<a href="https://colab.research.google.com/github/AdamPeetz/stable_diffusion/blob/main/latent_space_walks_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow keras_cv --upgrade --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 634.9/634.9 kB 10.0 MB/s eta 0:00:00


In [ ]:
import keras_cv
from tensorflow import keras
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import math
from PIL import Image
from tqdm import tqdm
import os, shutil 
from google.colab import drive 
drive.mount('/content/gdrive')
from matplotlib import pyplot

# Enable mixed precision
# (only do this if you have a recent NVIDIA GPU)
keras.mixed_precision.set_global_policy("mixed_float16")

# Instantiate the Stable Diffusion model
model = keras_cv.models.StableDiffusion(jit_compile=True)

You do not have Waymo Open Dataset installed, so KerasCV Waymo metrics are not available.
Mounted at /content/gdrive
By using this model checkpoint, you acknowledge that its usage is subject to the terms of the CreativeML Open RAIL-M license at https://raw.githubusercontent.com/CompVis/stable-diffusion/main/LICENSE


In [ ]:
seed = 123427
noise = tf.random.normal((512 // 8, 512 // 8, 4), seed=seed)

prompt_2 = "a psychedlic voyager, mindaltering, fractal, psytrance, high quality, masterpeice, highly detailed, elegant, sharp focus,concept art, character concepts, digital painting, scifi, fantasy, left of center, anatomically correct"
# prompt_1 =  "crystal city from above, aurora , gemstone, galaxy, mindaltering, fractal, psytrance, high quality, masterpeice, highly detailed, elegant, sharp focus,concept art, character concepts, digital painting, scifi, fantasy, anatomically correct"
prompt_1 = "planets,  fractal, psytrance, stars, galaxy, nebula, world, space, high quality, masterpeice, earthporn, highly detailed, elegant, sharp focus,concept art, character concepts, digital painting, scifi, fantasy, center frame, anatomically correct"

negative = "extra limbs, bad art, watermark, pencils, error, malformed, low detail, jpeg artifacts, cropped, plain background, ugly, low-res, poorly drawn face, out of frame, poorly drawn hands, blurry, bad art, extra hands, bad anatomy, amputee, missing limbs, amputated"

interpolation_steps = 256
batch_size = 1
batches = (interpolation_steps) // batch_size

encoding_1 = tf.squeeze(model.encode_text(prompt_1))
encoding_2 = tf.squeeze(model.encode_text(prompt_2))

interpolated_encodings = tf.linspace(encoding_1, encoding_2, interpolation_steps)

interpolated_encodings = tf.reshape(
    interpolated_encodings, (interpolation_steps, 77, 768)
)
batched_encodings = tf.split(interpolated_encodings, batches)

images = []
print(range(interpolation_steps))
for batch in range(interpolation_steps):
    print(batch)
    images.append(
      model.generate_image(
            batched_encodings[batch],
            negative_prompt=negative,
            batch_size=batch_size,
            diffusion_noise=noise,
        )
    )

492466864/492466864 [==============================] - 31s 0us/step
range(0, 256)
0
198180272/198180272 [==============================] - 12s 0us/step
1
50/50 [==============================] - 14s 271ms/step
2
50/50 [==============================] - 14s 276ms/step
3
50/50 [==============================] - 14s 271ms/step
4
50/50 [==============================] - 13s 265ms/step
5
50/50 [==============================] - 13s 266ms/step
6
50/50 [==============================] - 14s 271ms/step
7
50/50 [==============================] - 13s 270ms/step
8
50/50 [==============================] - 14s 273ms/step
9
50/50 [==============================] - 14s 274ms/step
10
50/50 [==============================] - 14s 271ms/step
11
50/50 [==============================] - 13s 268ms/step
12
50/50 [==============================] - 13s 266ms/step
13
50/50 [==============================] - 13s 268ms/step
14
50/50 [==============================] - 13s 267ms/step
15
50/50 [=====================

In [ ]:
counter = 0
for idx in tqdm(range(len(images))):
  plt.figure(figsize=(20, 20))
  plt.imshow(images[idx][0])
  plt.axis('off')
  filename = '/content/gdrive/My Drive/planegan/sample_output_2/generated_plot_%05d.png' % (counter+1)
  plt.savefig(filename, transparent=True,bbox_inches="tight",pad_inches=0.0 )
  plt.close()
  counter += 1

100%|██████████| 256/256 [02:56<00:00,  1.45it/s]


In [ ]:
import gc 
plt.clf()
plt.close()
gc.collect()

542067

In [ ]:
from google.colab import runtime
runtime.unassign()